# Model Definition and Evaluation
## Table of Contents
1. [Model Selection](#model-selection)
2. [Feature Engineering](#feature-engineering)
3. [Hyperparameter Tuning](#hyperparameter-tuning)
4. [Implementation](#implementation)
5. [Evaluation Metrics](#evaluation-metrics)
6. [Comparative Analysis](#comparative-analysis)


## Setup
### Imports

In [1]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import os, json, hashlib

from datetime import datetime
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.sequence import pad_sequences
from huggingface_hub import login, list_repo_files, hf_hub_download, upload_file

### Configuration

In [ ]:
REPO_ID = "mttfst/Paulette_Cloud_Tracks"
token = ""

WINDOW_SIZE = 30 #Minuten
CUTOFF_STEPS = 5

BATCH_SIZE = 8
EPOCHS = 2 #40

In [3]:
# =========================================
# Login HuggingFace
# =========================================
login(token)

### Load Data from HuggingFace

TRACK LENGTH FILE:

Diese CSV enthält Metadaten über alle Wolken:
- filename: Dateiname des Tracks
- track_len: Anzahl Zeitschritte (Lebensdauer)

WARUM WICHTIG?
- Wir trainieren nur auf Wolken mit >= 120 Zeitschritten
- Grund: Kurze Wolken (<60 Minuten) sind zu variabel/chaotisch
- Längere Wolken zeigen klare Lebenszyklen

DATENQUELLE:
Diese Datei wurde vorberechnet aus allen Tracks.
Spart Zeit beim Training-Setup.

In [14]:
tracks_120 = None

# Wenn lokal nicht da: von Hugging Face Dataset-Repo herunterladen
if tracks_120 is None:
    from huggingface_hub import hf_hub_download

    filename_in_repo = "track_len/track_len_exp_1.1.csv"

    print(f"⬇️  Downloading from HF: {REPO_ID}/{filename_in_repo}")
    local_file = hf_hub_download(
        repo_id=REPO_ID,
        repo_type="dataset",
        filename=filename_in_repo,
    )

    print(f"✅ Loading track_len from HF-cached file: {local_file}")
    tracks_120 = pd.read_csv(local_file)

print("tracks_120 shape:", tracks_120.shape)
tracks_120.head()

# =========================================
# Dataset-Split
# =========================================
files = list_repo_files(REPO_ID, repo_type="dataset")
csv_files = [f for f in files if f.startswith("exp_1.1/") and f.endswith(".csv")]

tracks_120 = tracks_120[tracks_120.track_len >= 120]
print("Total CSV tracks with at least 120 timesteps:", len(tracks_120))

tracks_120 = tracks_120.filename.to_list()

csv_files = [
    f for f in csv_files
    if f.split("/")[1] in tracks_120
]

random.seed(42)
random.shuffle(csv_files)

n = len(csv_files)
#train_files = csv_files[: int(0.7 * n)]
#val_files   = csv_files[int(0.7 * n): int(0.85 * n)]
#test_files  = csv_files[int(0.85 * n):]

train_files= csv_files[:75]
val_files= csv_files[100:125]
test_files= csv_files[150:175]

print(f"Train: {len(train_files)}, Val: {len(val_files)}, Test: {len(test_files)}")

⬇️  Downloading from HF: mttfst/Paulette_Cloud_Tracks/track_len/track_len_exp_1.1.csv
✅ Loading track_len from HF-cached file: /root/.cache/huggingface/hub/datasets--mttfst--Paulette_Cloud_Tracks/snapshots/8248ff1af359eabb4055f7d98300dd3810e359c3/track_len/track_len_exp_1.1.csv
tracks_120 shape: (9227, 2)
Total CSV tracks with at least 120 timesteps: 1115
Train: 75, Val: 25, Test: 25


## Model Selection

[Discuss the type(s) of models you consider for this task, and justify the selection.]



## Feature Engineering

[Describe any additional feature engineering you've performed beyond what was done for the baseline model.]


In [16]:
PROFILE_PREFIXES = ["qr_", "qc_", "qi_", "qs_", "qg_", "qv_", "roh_", "w_"]

SCALAR_FEATURES = [
    "cape_ml_L00", "cin_ml_L00",
    "lwp_L00",
    "iwp_L00", "rain_gsp_rate_L00",
    "tqc_L00", "tqi_L00", "area_m2"
]

In [17]:
def compute_remaining_lifetime(df, timestep_minutes=5):
    """
    Berechnet verbleibende Lebensdauer pro Zeitschritt
    """
    n = len(df)
    return [(n - i - 1) * timestep_minutes for i in range(n)]

#def compute_future_rain(df, timestep_minutes=5):
#    rain = df["rain_gsp_rate_L00"].values
#    dt = timestep_minutes * 60  # Sekunden

#    future_rain = []
#    for i in range(len(rain)):
#        future_rain.append(rain[i:].sum() * dt)

#    return future_rain

In [18]:
def extract_profile(df, prefix, n_levels=50):
    """
    Extrahiert ein vertikales Profil mit exakt n_levels.
    Fehlende Level werden mit 0 aufgefüllt.
    """
    data = np.zeros((len(df), n_levels), dtype="float32")
    for i in range(n_levels):
        col = f"{prefix}L{i:02d}"
        if col in df.columns:
            data[:, i] = df[col].values
    return data

In [31]:
def extract_ts_features_from_profiles(profiles, ):
    """
    profiles: (T, Z, F) mit F = len(PROFILE_PREFIXES)
    Rückgabe: (T, N_features)
    """
    idx = {name: i for i, name in enumerate(PROFILE_PREFIXES)}

    qc = profiles[:, :, idx["qc_"]]   # (T, Z)
    qi = profiles[:, :, idx["qi_"]]   # (T, Z)
    qr = profiles[:, :, idx["qr_"]]   # (T, Z)
    w  = profiles[:, :, idx["w_"]]    # (T, Z)

    T, Z = qc.shape
    levels = np.arange(Z, dtype="float32")

    ts_features = []

    for t in range(T):
        qc_t = qc[t]     # (Z,)
        qi_t = qi[t]
        qr_t = qr[t]
        w_t  = w[t]

        cloud_mask = (qc_t + qi_t) > 0

        if not np.any(cloud_mask):
            ts_features.append(np.zeros(12, dtype="float32"))
            continue

        # Level-Indizes mit Wolke
        cloud_levels = levels[cloud_mask]

        cloud_top  = cloud_levels.min()   # kleinster Index
        cloud_base = cloud_levels.max()   # größter Index
        cloud_thickness = cloud_base - cloud_top

        cloud_mass = np.sum(qc_t + qi_t)
        rain_mass  = np.sum(qr_t)

        w_in_cloud = w_t[cloud_mask]
        mean_w = np.mean(w_in_cloud)
        max_w  = np.max(w_in_cloud)

        height_max_qc = np.argmax(qc_t)   # Index des Maximums
        height_max_w  = np.argmax(w_t)

        center_of_mass = np.sum(levels * (qc_t + qi_t)) / (cloud_mass + 1e-12)

        ts_features.append([
            cloud_base,
            cloud_top,
            cloud_thickness,
            cloud_mass,
            rain_mass,
            mean_w,
            max_w,
            height_max_qc,
            height_max_w,
            center_of_mass,
            np.max(qc_t),
            np.std(w_in_cloud),
        ])

    return np.array(ts_features, dtype="float32")


In [20]:
def aggregate_ts_features(ts_features, max_minutes=20):
    """
    Aggregiert Zeitreihenfeatures über die ersten max_minutes
    """
    n_steps = int(max_minutes / 0.5)
    X = ts_features[:n_steps]

    agg = []

    # pro Feature: mean, std, min, max, slope
    t = np.arange(len(X))

    for i in range(X.shape[1]):
        xi = X[:, i]

        agg.extend([
            np.mean(xi),
            np.std(xi),
            np.min(xi),
            np.max(xi),
            np.polyfit(t, xi, 1)[0] if len(xi) > 1 else 0.0
        ])

    return np.array(agg, dtype="float32")


In [32]:
def preprocess_cloud(df):
    df = df.sort_values("time")

    # Target
    y_lifetime = compute_remaining_lifetime(df)

    # Profile (T, Z, F)
    profile_features = []
    for prefix in PROFILE_PREFIXES:
        prof = extract_profile(df, prefix, n_levels=50)
        profile_features.append(prof)
    
    profiles = np.stack(profile_features, axis=-1)
    
    # Scalars
    scalars = df[SCALAR_FEATURES].values.astype("float32")
    
    # 🔥 neue TS-Features
    ts_features = extract_ts_features_from_profiles(profiles)

    # 🔥 Aggregation
    agg_features = aggregate_ts_features(
        ts_features,
        max_minutes=WINDOW_SIZE*2
    )

    return {
        "ts_features": ts_features,      # (T, 12)
        "scalars": scalars,              # (T, 8)
        "agg_features": agg_features,    # (60,)
        "y": np.array(y_lifetime, dtype="float32")[:, None]
    }

In [33]:
def load_and_preprocess(files):
    samples = []
    for f in files:
        local_file = hf_hub_download(
            repo_id=REPO_ID,
            repo_type="dataset",
            filename=f,
        )
        df = pd.read_csv(local_file)
        
        if len(df) <= CUTOFF_STEPS:
            continue
        
        sample = preprocess_cloud(df)
        samples.append(sample)
    
    return samples

In [34]:
print("\n Loading data...")
train_samples = load_and_preprocess(train_files)
val_samples   = load_and_preprocess(val_files)
test_samples  = load_and_preprocess(test_files)

print(f"Loaded: {len(train_samples)} train, {len(val_samples)} val, {len(test_samples)} test")


 Loading data...
Loaded: 75 train, 25 val, 25 test


In [38]:
sample = train_samples[8]
ts = sample["ts_features"]  
ts_cols = [
    "cloud_base", "cloud_top", "cloud_thickness",
    "cloud_mass", "rain_mass",
    "mean_w", "max_w",
    "height_max_qc", "height_max_w",
    "center_of_mass", "max_qc", "std_w_in_cloud"
]

ts_df = pd.DataFrame(ts, columns=ts_cols)
ts_df.head(15)

,cloud_base,cloud_top,cloud_thickness,cloud_mass,rain_mass,mean_w,max_w,height_max_qc,height_max_w,center_of_mass,max_qc,std_w_in_cloud
0,41.0,4.0,37.0,0.008732,0.002572,0.233418,1.020461,30.0,36.0,32.851341,0.001669,0.409249
1,42.0,4.0,38.0,0.008874,0.002705,0.242999,1.030474,30.0,36.0,32.879936,0.001720,0.409450
2,42.0,4.0,38.0,0.008977,0.002832,0.246419,1.034724,30.0,36.0,32.910141,0.001759,0.412110
3,42.0,4.0,38.0,0.009041,0.002950,0.248115,1.033336,30.0,36.0,32.938213,0.001787,0.412336
4,42.0,4.0,38.0,0.009063,0.003056,0.247676,1.024991,30.0,36.0,32.962696,0.001802,0.409315
5,41.0,4.0,37.0,0.009367,0.005843,0.272165,1.041178,28.0,34.0,31.107204,0.001786,0.386302
6,41.0,4.0,37.0,0.010109,0.006672,0.298150,1.132603,28.0,34.0,31.141527,0.001956,0.422921
7,42.0,4.0,38.0,0.010749,0.007590,0.322319,1.220246,28.0,34.0,31.183786,0.002097,0.452325
8,42.0,4.0,38.0,0.011284,0.008600,0.345116,1.302683,28.0,34.0,31.222240,0.002220,0.485179
9,42.0,4.0,38.0,0.011693,0.009699,0.367592,1.382468,28.0,33.0,31.252686,0.002323,0.516288


## Hyperparameter Tuning

[Discuss any hyperparameter tuning methods you've applied, such as Grid Search or Random Search, and the rationale behind them.]


In [ ]:
# Implement hyperparameter tuning
# Example using GridSearchCV with a DecisionTreeClassifier
# param_grid = {'max_depth': [2, 4, 6, 8]}
# grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5)
# grid_search.fit(X_train, y_train)


## Implementation

[Implement the final model(s) you've selected based on the above steps.]


In [ ]:
# Implement the final model(s)
# Example: model = YourChosenModel(best_hyperparameters)
# model.fit(X_train, y_train)


## Evaluation Metrics

[Clearly specify which metrics you'll use to evaluate the model performance, and why you've chosen these metrics.]


In [ ]:
# Evaluate the model using your chosen metrics
# Example for classification
# y_pred = model.predict(X_test)
# print(classification_report(y_test, y_pred))

# Example for regression
# mse = mean_squared_error(y_test, y_pred)

# Your evaluation code here


## Comparative Analysis

[Compare the performance of your model(s) against the baseline model. Discuss any improvements or setbacks and the reasons behind them.]


In [ ]:
# Comparative Analysis code (if applicable)
# Example: comparing accuracy of the baseline model and the new model
# print(f"Baseline Model Accuracy: {baseline_accuracy}, New Model Accuracy: {new_model_accuracy}")
